In [57]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


In [58]:
def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")
    return dataframe

In [59]:
# Function for cleaning comma seperated string values of a series into list of strings. Accepts one series.

def clean_comma_series(Series):
    Series = Series.str.replace("_", " ")
    Series = Series.str.title()
    Series = Series.str.split(",")
    return Series

In [60]:
file_path = '/Users/claire/git/flatiron/phase_02/finalproject/DSC-Phase2-Project2/unzippedData/im.db'
conn = sqlite3.connect(file_path)


In [61]:
! ls ../unzippedData

bom.movie_gross.csv.gz  rt.movie_info.tsv.gz    tn.movie_budgets.csv.gz
im.db                   rt.reviews.tsv.gz
im.db.zip               tmdb.movies.csv.gz


In [62]:
cursor = conn.cursor()

![Image of ERD Diagram IMDB](https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project-v3/main/movie_data_erd.jpeg)


In [63]:
# Create a dataframe from the master table of our sql database:
# Write the query:
imdb_query = """ 
    SELECT *
    FROM sqlite_master
"""
# Read the contents of the query into a Pandas dataframe:
imdb_df = pd.read_sql(imdb_query, conn)

# From the pandas dataframe, iterate through the column "name" (all our table names)
# Make a list of those table names:
table_list = [name for name in imdb_df['name']]

def table_to_df(my_table, my_conn):
    '''Input is name of database table as a string, output is dataframe'''
    my_query = f"""SELECT * FROM {my_table}"""
    df = pd.read_sql(my_query, my_conn)
    return df

# Make a dataframe where the keys are the names of the tables, 
# and the values are pandas dataframes made from those tables:
dict_tables = {name: pd.DataFrame(data=table_to_df(name,conn)) for name in table_list}

# Make dataframe names, prefixed by "df_" and then the original table name
# Output is 
for i in range(len(table_list)):
    # converts the STRING f"df_{table_list[i]}" to a VARIABLE equal to each df in df_tables
    exec(f"{table_list[i]}_df" + " = dict_tables[table_list[i]]") 
    

# Lez try out one of our brand-new dfs!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
writers_df.head()

####### We now have these dataframes:
"""
movie_basics_df
directors_df
known_for_df
movie_akas_df
movie_ratings_df
persons_df
principals_df
writers_df
"""

'\nmovie_basics_df\ndirectors_df\nknown_for_df\nmovie_akas_df\nmovie_ratings_df\npersons_df\nprincipals_df\nwriters_df\n'

In [64]:
# Clean persons_df
persons_df.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [65]:
persons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


In [66]:
# Nick's data cleaning
# persons_df.drop(persons_df[persons_df['death_year'].isna()==True].index, inplace=True)
# persons_df.drop(persons_df[persons_df['birth_year']<1930].index, inplace=True)

In [67]:
persons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


In [68]:
persons_df['birth_year'].describe()

count    82736.000000
mean      1967.043826
std         22.122190
min          1.000000
25%       1957.000000
50%       1971.000000
75%       1981.000000
max       2014.000000
Name: birth_year, dtype: float64

In [69]:
# First we should create columns for their professions, and THEN we decide how to filter birth and death year

In [71]:
persons_df = persons_df['primary_profession'].str.get_dummies(',')

In [72]:
persons_df.head()

,actor,actress,animation_department,art_department,art_director,assistant,assistant_director,camera_department,casting_department,casting_director,...,publicist,set_decorator,sound_department,soundtrack,special_effects,stunts,talent_agent,transportation_department,visual_effects,writer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [73]:
persons_df.describe()

,actor,actress,animation_department,art_department,art_director,assistant,assistant_director,camera_department,casting_department,casting_director,...,publicist,set_decorator,sound_department,soundtrack,special_effects,stunts,talent_agent,transportation_department,visual_effects,writer
count,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,...,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.000000,606648.00000,606648.000000
mean,0.293149,0.156707,0.004053,0.014692,0.007621,0.000119,0.026236,0.065056,0.004619,0.003951,...,0.000137,0.002365,0.025188,0.034201,0.003059,0.007103,0.000516,0.001109,0.01020,0.233256
std,0.455206,0.363525,0.063537,0.120318,0.086963,0.010894,0.159836,0.246625,0.067805,0.062734,...,0.011696,0.048578,0.156695,0.181745,0.055228,0.083979,0.022709,0.033289,0.10048,0.422904
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [74]:
persons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 37 columns):
 #   Column                     Non-Null Count   Dtype
---  ------                     --------------   -----
 0   actor                      606648 non-null  int64
 1   actress                    606648 non-null  int64
 2   animation_department       606648 non-null  int64
 3   art_department             606648 non-null  int64
 4   art_director               606648 non-null  int64
 5   assistant                  606648 non-null  int64
 6   assistant_director         606648 non-null  int64
 7   camera_department          606648 non-null  int64
 8   casting_department         606648 non-null  int64
 9   casting_director           606648 non-null  int64
 10  cinematographer            606648 non-null  int64
 11  composer                   606648 non-null  int64
 12  costume_department         606648 non-null  int64
 13  costume_designer           606648 non-null  int64
 14  dire

In [75]:
movie_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [76]:
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
